In [2]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
load_dotenv(dotenv_path='./../.env')

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study"

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.3,
    model="gpt-4o",
)

In [40]:
long_query = """
Python 강의 중 동적 타이핑(Dynamic Typing)에 대한 부분만 찾아줘
"""

In [4]:
from langchain_core.pydantic_v1 import BaseModel, Field

class YoutubeQuery(BaseModel):
    youtube_query: str = Field(description="youtube에서 사용할 검색어")
    llm_query: str = Field(description="YouTube 컨텐츠가 담고있는 Text 정보에서 사용자가 원하는 부분을 찾기 위한 자연어 질의")
    language: str = Field(description="검색할 컨텐츠의 한글, 영어 구분(ko, en 둘중에 하나만 가능)")

In [5]:
from langchain.output_parsers import PydanticOutputParser

parser_query = PydanticOutputParser(pydantic_object=YoutubeQuery)

In [6]:
youtube_search_query_tips ="""
YouTube 검색 시 query 작성에 대한 주요 규칙과 팁:

1. 기본 검색:
   - 단순히 키워드나 문구를 입력합니다.
   - 예: `Python tutorial`

2. 정확한 구문 검색:
   - 큰따옴표로 묶어 정확한 구문을 검색합니다.
   - 예: `"Python for beginners"`

3. 불리언 연산자:
   - AND: 기본적으로 모든 단어는 AND로 연결됩니다.
   - OR: 단어 사이에 OR을 사용합니다.
   - 예: `Python OR Java tutorial`

4. 제외 검색:
   - 빼고 싶은 단어 앞에 마이너스(-) 기호를 사용합니다.
   - 예: `Python tutorial -advanced`

5. 와일드카드:
   - 별표(*)를 사용해 부분 일치를 검색할 수 있습니다.
   - 예: `Python * tutorial`

6. 채널 검색:
   - 채널명 앞에 @를 붙입니다.
   - 예: `@GoogleDevelopers Python`

7. 필터 사용:
   - 특정 필터를 쿼리에 직접 포함할 수 있습니다.
   - 예: `Python tutorial after:2023-01-01` (2023년 이후 영상)

8. 대소문자 구분:
   - YouTube 검색은 대소문자를 구분하지 않습니다.

9. 특수 문자:
   - 대부분의 특수 문자는 무시됩니다.

10. 길이 제한:
    - 쿼리 문자열에는 길이 제한이 있으므로 너무 길지 않게 작성합니다.

11. 언어와 지역:
    - 검색 결과는 사용자의 위치와 언어 설정에 영향을 받을 수 있습니다.

12. 동의어:
    - YouTube는 때때로 동의어를 자동으로 포함시킵니다.

이러한 규칙을 조합하여 더 정확하고 효과적인 검색 쿼리를 만들 수 있습니다. 예를 들어:

```python
query = '"Python for beginners" -advanced after:2023-01-01'
```

이 쿼리는 "Python for beginners"라는 정확한 구문을 포함하고, "advanced"를 제외하며, 2023년 1월 1일 이후에 업로드된 비디오를 검색합니다.
"""

In [7]:
from langchain.prompts import PromptTemplate

prompt_query = PromptTemplate.from_template(
"""
System :
너는 사용자가 원하는 youtube 컨텐츠를 찾아주는 AI야.

사용자의 query를 분석해서 다음 3가지 정보를 생성해줘.
 - 사용자가 원하는 YouTube 컨텐츠를 찾을 수 있도록 YouTube 검색어를 만들어줘.
 - 찾아야 할 컨텐츠가 한글인 경우 'ko' 영어인 경우 'en'이라고 표시해줘.
 - YouTube 컨텐츠가 담고있는 Text 정보에서 사용자가 원하는 부분을 찾기 위한 자연어 질의를 만들어줘
   예를 들어, 채널명이나 필터 정보 등 YouTube 검색에 사용되는 내용은 제외해야되.
   
검색어를 만들때 tips 를 참고해.

tips :
{tips}

query : 
{query}

Format :
{format}
"""
)

# format에 PydanticOutputParser의 format 추가
prompt_query = prompt_query.partial(
    format = parser_query.get_format_instructions(),
    tips = youtube_search_query_tips,
    )

In [8]:
chain_1st = prompt_query | llm | parser_query

In [28]:
chain_1st.output_schema.schema()

{'title': 'YoutubeQuery',
 'type': 'object',
 'properties': {'youtube_query': {'title': 'Youtube Query',
   'description': 'youtube에서 사용할 검색어',
   'type': 'string'},
  'llm_query': {'title': 'Llm Query',
   'description': 'YouTube 컨텐츠가 담고있는 Text 정보에서 사용자가 원하는 부분을 찾기 위한 자연어 질의',
   'type': 'string'},
  'language': {'title': 'Language',
   'description': '검색할 컨텐츠의 한글, 영어 구분(ko, en 둘중에 하나만 가능)',
   'type': 'string'}},
 'required': ['youtube_query', 'llm_query', 'language']}

#### [TEST]. 첫번째 chain TEST

In [10]:
# Test
response_1st = chain_1st.invoke({"query":long_query})
response_1st

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


YoutubeQuery(youtube_query='삼성SDS Fabrix LLM 서비스 소개 after:2023-07-01', llm_query='삼성SDS의 Fabrix(패브릭스)라는 LLM 서비스에 대한 소개', language='ko')

c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings


In [15]:
from youtube import ContentManager, Content, YouTube
from functions import get_youtube_captions

def results_from_youtube(response_1st):

    print(response_1st)
    language = response_1st.language
    youtube_query = response_1st.youtube_query
    llm_query = response_1st.llm_query

    yt = YouTube()

    articles = yt.search(
        query=youtube_query, 
        caption=True, 
        caption_language=language
    )

    c_manager = ContentManager()

    for a in articles:
        c_manager.add(Content.from_dict(a))

    for c in c_manager:
        c_manager.add_captions(c.id, get_youtube_captions(c.id, language=language))

    youtube_captions = [   
        {
            "query": llm_query,
            "captions": c_manager[content.id].captions,
            "video_id": content.id,
        } for content in c_manager
    ]

    return youtube_captions

#### [TEST]. YouTube에서 자막 가져오기

In [16]:
# Test
youtube_captions = results_from_youtube(response_1st)
youtube_captions

2024-07-17 14:26:54,952 - INFO - file_cache is only supported with oauth2client<4.0.0
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\prof\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host '70.10.15.10'. Adding certificate verification is strongly ad

[{'query': '삼성SDS의 Fabrix(패브릭스)라는 LLM 서비스에 대한 소개',
  'captions': [{'text': '[음악]', 'start': 0.18, 'duration': 13.48},
   {'text': '안녕하십니까 저는\n삼성SDS에서', 'start': 15.059, 'duration': 3.181},
   {'text': 'AI와 클라우드 사업을 담당하고 있는\n구형준입니다 반갑습니다',
    'start': 18.24,
    'duration': 4.34},
   {'text': '다들 리스크를 말씀하시는데 제가\n네번째고 김대식 교수님 아직 계시는데',
    'start': 24.66,
    'duration': 5.64},
   {'text': '되게 재밌어서 제가 한 서너 번 웃은 것 같습니다', 'start': 30.3, 'duration': 4.62},
   {'text': '제가 발표를 웃게 해드릴지는 모르겠지만\n저희 대표님', 'start': 34.92, 'duration': 3.12},
   {'text': '어디 계시죠? 저희 또 대표님 너무\n똑똑하셔서 똑 부러지게 또 발표를 잘',
    'start': 38.04,
    'duration': 4.199},
   {'text': '하시고요. 제 바로 전에 했던\n송부사님은 저희 회사에서 유명합니다',
    'start': 42.239,
    'duration': 4.861},
   {'text': '교수 같이 말 잘하고 이 프리젠테이션\n잘한다고 제가 할 수 있는 거는 얼마',
    'start': 47.1,
    'duration': 3.6},
   {'text': '없지만 한번 웃으면서\n좀 재밌게 하려고 노력을', 'start': 50.7, 'duration': 3.72},
   {'text': '해보겠습니다. 지금 세션을 다\n들어보시면 저희 SDS에서 생각하고',
    'start': 54.42,
    'duration': 

In [17]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

class Segment(BaseModel):
    title: str = Field(description="해당 구간의 제목 (20자 이하)")
    start: float = Field(description="해당 구간의 시작 시간(초)")
    duration: float = Field(description="해당 구간의 duration (세부 구간들의 duration 합계)(초))")

class Summary(BaseModel):
    video_id: str = Field(description="동영상을 식별하는 id")
    description: str = Field(description="captions 내용 요약")
    segments: List[Segment] = Field(description="사용자 query를 만족하는 구간들 정보")

In [18]:
from langchain.output_parsers import PydanticOutputParser

parser_selection = PydanticOutputParser(pydantic_object=Summary)

In [19]:
from langchain.prompts import PromptTemplate

prompt_selection = PromptTemplate.from_template(
"""
System :
너는 동영상 편집자이다.
먼저 captions 내용을 100자 이내로 요약해.
요약 결과는 네가 시청자에게 동영상을 소개하는 말투여야 해.
그리고 captions에서 사용자 query를 잘 설명하고 있는 내용을 뽑아내고
그 내용이 포함된 구간을 찾아내.
여기서 구간이란 captions 내의 하나의 세부 구간이 아니라 연속되는 여러개의 세부 구간의 합을 의미한다.

예) 

세부 구간 1 :
  start : 112.0, duration : 5.0
세부 구간 2 :
  start : 117.0, duration : 4.0
세부 구간 3 :
  start : 121.0, duration : 6.0
세부 구간 4 : 
  start : 127.0, duration : 3.0

네가 찾은 하나의 구간이 세부 구간 1 ~ 4 인 경우
구간의 start, duration은 다음과 같이 계산 하면 된다.
  start : 112.0 (세부 구간 1 의 start)
  duration : 18.0 ( 5.0 + 4.0 + 6.0 + 3.0, 세부 구간 4개 duration 의 합계)

해당 구간의 내용을 20자 이내로 요약해서 구간 제목을 만들어.
그 구간은 없을 수도 있고 여러개 일수도 있다.
그 구간이 4개 이상이라면 가장 적합한 것 3개만 찾아내라.
video_id 값은 동영상을 식별하는 id이다. 변경하지 말고 그대로 출력하라.

query : 
{query}

video_id :
{video_id}

captions : 
{captions}

Format :
{format}
"""
)

# format에 PydanticOutputParser의 format 추가
prompt_selection = prompt_selection.partial(format=parser_selection.get_format_instructions())

In [20]:
chain_2nd = prompt_selection | llm | parser_selection

In [27]:
chain_2nd.output_schema.schema()

{'title': 'Summary',
 'type': 'object',
 'properties': {'video_id': {'title': 'Video Id',
   'description': '동영상을 식별하는 id',
   'type': 'string'},
  'description': {'title': 'Description',
   'description': 'captions 내용 요약',
   'type': 'string'},
  'segments': {'title': 'Segments',
   'description': '사용자 query를 만족하는 구간들 정보',
   'type': 'array',
   'items': {'$ref': '#/definitions/Segment'}}},
 'required': ['video_id', 'description', 'segments'],
 'definitions': {'Segment': {'title': 'Segment',
   'type': 'object',
   'properties': {'title': {'title': 'Title',
     'description': '해당 구간의 제목 (20자 이하)',
     'type': 'string'},
    'start': {'title': 'Start',
     'description': '해당 구간의 시작 시간(초)',
     'type': 'number'},
    'duration': {'title': 'Duration',
     'description': '해당 구간의 duration (세부 구간들의 duration 합계)(초))',
     'type': 'number'}},
   'required': ['title', 'start', 'duration']}}}

#### [TEST]. YouTube 자막에서 Query에 적합한 부분 골라내기

In [29]:
temp_responses = chain_2nd.batch(youtube_captions)
temp_responses

2024-07-17 15:03:50,110 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:03:50,567 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:03:50,587 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:03:51,497 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:03:52,637 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Summary(video_id='y0PtRm3KZHU', description='삼성SDS의 FabriX 서비스와 LLM을 통한 하이퍼 오토메이션 소개', segments=[Segment(title='FabriX 소개', start=254.099, duration=3.841), Segment(title='FabriX와 LLM 결합', start=366.0, duration=26.459), Segment(title='FabriX의 기능 및 활용', start=476.28, duration=81.0)]),
 Summary(video_id='f0j-nROVHZg', description='삼성SDS의 Fabrix 서비스로 기업의 하이퍼 오토메이션을 가속화하는 방법을 소개합니다.', segments=[Segment(title='Fabrix 서비스 소개', start=31.439, duration=15.8), Segment(title='코파일럿 기능 설명', start=240.56, duration=93.68), Segment(title='SCM 코파일럿 데모', start=541.88, duration=117.52)]),
 Summary(video_id='7pI9nZrfZSY', description='삼성SDS의 Fabrix 서비스로 업무생산성 향상을 목표로 합니다.', segments=[Segment(title='업무생산성 향상 목표', start=17.84, duration=17.0)]),
 Summary(video_id='K8Ys_0BH-xA', description='삼성SDS의 Fabrix 서비스와 협력 모델 소개', segments=[Segment(title='클라우드 서비스 협력', start=0.08, duration=9.24), Segment(title='한글 모델과 협력', start=17.56, duration=13.639), Segment(title='SCP 플랫폼 구축', start=31.279, duration=9.6)]),
 Summar

In [50]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

full_chain = (
    {"query": RunnablePassthrough()} | 
    chain_1st |
    RunnableLambda(results_from_youtube) |
    chain_2nd.map()
)

2024-07-17 15:53:30,914 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:53:30,925 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-07-17 15:53:41,301 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:53:43,049 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:53:43,191 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:53:43,907 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 15:53:44,350 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### X. 쿼리와 쿼리 실행

In [58]:
long_query = """
LF소나타 전조등 교체하는 법 알려줘
"""
responses = full_chain.invoke(long_query)

2024-07-17 16:19:17,955 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 16:19:17,965 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-07-17 16:19:31,098 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 16:19:31,471 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 16:19:31,578 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 16:19:33,217 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-17 16:19:35,452 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [59]:
responses

[Summary(video_id='jlm9Yu39elE', description='LF소나타 전조등 교체 방법을 단계별로 설명합니다.', segments=[Segment(title='전구 커버 제거 및 커넥터 분리', start=15.33, duration=26.4), Segment(title='전구 교체 및 재조립', start=94.23, duration=41.558)]),
 Summary(video_id='gDtlgQReIA4', description='LF소나타 전조등을 LED로 교체하는 과정을 자세히 설명합니다.', segments=[Segment(title='전조등 교체 준비', start=78.33, duration=4.02), Segment(title='배선 작업', start=109.12, duration=24.54), Segment(title='조사각 조절', start=172.22, duration=31.64)]),
 Summary(video_id='nvSjziBHwTw', description='LF소나타 전조등을 LED로 교체하는 방법을 설명합니다.', segments=[Segment(title='LED로 전조등 교체', start=24.569, duration=8.88), Segment(title='조사각 조절', start=38.967, duration=6.44), Segment(title='작업 마무리', start=53.028, duration=5.03)]),
 Summary(video_id='4GLhc4aJJtQ', description='쏘나타 뉴라이즈 LED DRL 장착 및 전조등 교체 과정을 소개합니다.', segments=[Segment(title='작업 준비 및 부품 소개', start=30.34, duration=14.74), Segment(title='범퍼 탈거 및 DRL 장착', start=48.19, duration=10.09), Segment(title='커넥터 변환 및 조립', start=65.18, dura

### X. 동영상 제작
1. youtube 컨텐츠 download -> mp4 파일 생성
2. 영상 slicing -> segment 영상 파일 생성
3. 영상에 title 삽입
4. segment별 설명 영상 생성
5. youtube 컨텐츠 별 설명 영상 생성
6. 영상들 이어 붙이기

In [60]:
import uuid
job_id = str(uuid.uuid4())

In [61]:
from functions import download_youtube

for summary in responses:

    if not summary.segments:
        continue
    
    youtube_url = f"https://www.youtube.com/watch?v={summary.video_id}"

    file_path, file_name = download_youtube(youtube_url, job_id=job_id)
    print(f"Downloaded file names : {file_path}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=jlm9Yu39elE
[youtube] jlm9Yu39elE: Downloading webpage
[youtube] jlm9Yu39elE: Downloading ios player API JSON
[youtube] jlm9Yu39elE: Downloading m3u8 information
[info] jlm9Yu39elE: Downloading 1 format(s): 18
[download] Destination: 3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE.mp4
[download] 100% of   11.01MiB in 00:00:04 at 2.52MiB/s     
Downloaded file names : c:\GitHub\rorry\langchain_study\youtube_summary\3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE.mp4
[youtube] Extracting URL: https://www.youtube.com/watch?v=gDtlgQReIA4
[youtube] gDtlgQReIA4: Downloading webpage
[youtube] gDtlgQReIA4: Downloading ios player API JSON
[youtube] gDtlgQReIA4: Downloading m3u8 information
[info] gDtlgQReIA4: Downloading 1 format(s): 18
[download] Destination: 3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4.mp4
[download] 100% of   20.94MiB in 00:00:14 at 1.45MiB/s     
Downloaded file names : c:\GitHub\rorry\langchain_study\youtube_s

In [62]:
from functions import slice_video

for summary in responses:

    content_id = job_id + "_" + summary.video_id

    segments = [(s.start, s.duration, content_id+"_"+str(i)+".mp4") for i, s in enumerate(summary.segments)]
    
    results = slice_video(content_id+".mp4", segments, height=540, width=960)


2024-07-17 16:23:00,266 - INFO - Successfully created segment: 3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_0.mp4
2024-07-17 16:23:07,585 - INFO - Successfully created segment: 3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_1.mp4
2024-07-17 16:23:08,735 - INFO - Successfully created segment: 3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_0.mp4
2024-07-17 16:23:13,164 - INFO - Successfully created segment: 3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_1.mp4
2024-07-17 16:23:18,378 - INFO - Successfully created segment: 3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_2.mp4
2024-07-17 16:23:19,994 - INFO - Successfully created segment: 3037f672-ed8c-431a-b1b5-1e681c867b32_nvSjziBHwTw_0.mp4
2024-07-17 16:23:21,498 - INFO - Successfully created segment: 3037f672-ed8c-431a-b1b5-1e681c867b32_nvSjziBHwTw_1.mp4
2024-07-17 16:23:22,670 - INFO - Successfully created segment: 3037f672-ed8c-431a-b1b5-1e681c867b32_nvSjziBHwTw_2.mp4
2024-07-17 16:23:25,555 - INFO - Successfully created se

In [63]:
from functions import Position, draw_text, add_text_to_image, create_video_with_audio_and_image

segment_list = []

for summary in responses:

    text_config = dict(
        position = Position.CENTER,
        font_color=(0, 128, 255),
        font_size=60,
        text = summary.description,
    )

    content_id = job_id + "_" + summary.video_id

    add_text_to_image("bg_image.png", content_id+".png", **text_config)

    create_video_with_audio_and_image(
        content_id+".png", 
        content_id+"_c.mp4", 
        text = summary.description,
        width = 640,
        height = 360,
    )

    segment_list.append(content_id+"_c.mp4")

    for i, s in enumerate(summary.segments):

        segment_id = content_id + "_" + str(i)

        text_config = dict(
            position = Position.CENTER,
            font_color=(0, 0, 255),
            font_size=80,
            text = s.title,
        )

        add_text_to_image("bg_image.png", segment_id+".png", **text_config)

        create_video_with_audio_and_image(
            segment_id+".png", 
            segment_id+"_c.mp4", 
            text = s.title,
            width = 640,
            height = 360,
            )
        
        segment_list.append(segment_id+"_c.mp4")

        text_config.update(
            position = Position.BOTTOM,
            color = (255, 0, 0),
            font_size = 72,
        )

        draw_text(segment_id+".mp4", [(0, s.duration, text_config)])

        segment_list.append(segment_id+"_t.mp4")


2024-07-17 16:24:01,378 - INFO - Starting text overlay process for 3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_0.mp4
2024-07-17 16:24:01,379 - INFO - Starting ffmpeg process
2024-07-17 16:24:06,190 - INFO - Text overlay completed. Output saved as 3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_0_t.mp4
2024-07-17 16:24:07,431 - INFO - Starting text overlay process for 3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_1.mp4
2024-07-17 16:24:07,432 - INFO - Starting ffmpeg process
2024-07-17 16:24:15,673 - INFO - Text overlay completed. Output saved as 3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_1_t.mp4
2024-07-17 16:24:18,121 - INFO - Starting text overlay process for 3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_0.mp4
2024-07-17 16:24:18,122 - INFO - Starting ffmpeg process
2024-07-17 16:24:19,308 - INFO - Text overlay completed. Output saved as 3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_0_t.mp4
2024-07-17 16:24:20,400 - INFO - Starting text overlay process for 3037f6

In [64]:
segment_list

['3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_0_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_0_t.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_1_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_jlm9Yu39elE_1_t.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_0_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_0_t.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_1_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_1_t.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_2_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_gDtlgQReIA4_2_t.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_nvSjziBHwTw_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_nvSjziBHwTw_0_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_nvSjziBHwTw_0_t.mp4',
 '3037f672-ed8c-431a-b1b5-1e681c867b32_nvSjziBHwTw_1_c.mp4',
 '3037f672-ed8c-431a-b1b5-1e68

In [57]:
from functions import concatenate_videos

#output_file = "삼성SDS_FabriX.mp4"
output_file = "LF소나타_전조등교체방법.mp4"

concatenate_videos(segment_list, output_file, width=640 , height=360 )